In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
from torch.utils.data import DataLoader

In [4]:
# 데이터 전처리 및 데이터 증강
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  # 무작위로 잘라서 일반화
    transforms.RandomHorizontalFlip(),    # 랜덤 좌우반전
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # 색상 변화
    transforms.RandomAdjustSharpness(2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 데이터셋 로드
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# 개선된 CNN 모델 정의
class ImprovedCNN(nn.Module):
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.4)
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 100)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))  # 32x32 -> 16x16
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))  # 16x16 -> 8x8
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))  # 8x8 -> 4x4
        x = self.dropout(x)
        x = x.view(-1, 256 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 생성
model = ImprovedCNN()

# 손실 함수 및 최적화 기법
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# 훈련 함수
def train(model, trainloader, criterion, optimizer, scheduler, num_epochs=30):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(trainloader, 0):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            if i % 100 == 99:
                print(f"Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.3f}")
                running_loss = 0.0

        scheduler.step()

# 테스트 함수
def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy on test set: {100 * correct / total:.2f}%')

# 모델 학습 및 평가
train(model, trainloader, criterion, optimizer, scheduler, num_epochs=30)
test(model, testloader)

Epoch 1, Batch 100, Loss: 4.345
Epoch 1, Batch 200, Loss: 3.952
Epoch 1, Batch 300, Loss: 3.789
Epoch 1, Batch 400, Loss: 3.647
Epoch 1, Batch 500, Loss: 3.533
Epoch 1, Batch 600, Loss: 3.462
Epoch 1, Batch 700, Loss: 3.300
Epoch 2, Batch 100, Loss: 3.125
Epoch 2, Batch 200, Loss: 3.121
Epoch 2, Batch 300, Loss: 3.078
Epoch 2, Batch 400, Loss: 3.038
Epoch 2, Batch 500, Loss: 3.003
Epoch 2, Batch 600, Loss: 2.939
Epoch 2, Batch 700, Loss: 2.915
Epoch 3, Batch 100, Loss: 2.790
Epoch 3, Batch 200, Loss: 2.749
Epoch 3, Batch 300, Loss: 2.805
Epoch 3, Batch 400, Loss: 2.744
Epoch 3, Batch 500, Loss: 2.733
Epoch 3, Batch 600, Loss: 2.701
Epoch 3, Batch 700, Loss: 2.678
Epoch 4, Batch 100, Loss: 2.578
Epoch 4, Batch 200, Loss: 2.597
Epoch 4, Batch 300, Loss: 2.583
Epoch 4, Batch 400, Loss: 2.535
Epoch 4, Batch 500, Loss: 2.535
Epoch 4, Batch 600, Loss: 2.485
Epoch 4, Batch 700, Loss: 2.516
Epoch 5, Batch 100, Loss: 2.412
Epoch 5, Batch 200, Loss: 2.462
Epoch 5, Batch 300, Loss: 2.411
Epoch 5,